**AI-Powered Market Trend Analysis for Amazon Products**

In [ ]:
%pip install kagglehub

import kagglehub # type: ignore

path = kagglehub.dataset_download("ssrai7/amazon-global-sales-and-review-data")

print("Path to dataset files:", path)

In [ ]:
import os

for filename in os.listdir(path):
    print(filename)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

excel_file_path = os.path.join(path, "Amazon_Combined_Data.xlsx")
df = pd.read_excel(excel_file_path)
display(df.head(10))

In [ ]:
df.info()


Referings: We can use Price and Number of reviews for regression analysis, while
Product category and all other objects in classification tasks

In [ ]:
df.describe()

In [ ]:
# Splitting Shipment-> 'Shipment Location' and 'Shipment Date'
df[['Shipment Location', 'Shipment Date']] = df['Shipment'].str.split(' ', n=3, expand=True).iloc[:, [2, 3]]

df['Shipment Date'] = pd.to_datetime(df['Shipment Date'], errors='coerce')


In [ ]:
price_bins = [0, 20, 50, 100, 200, 500, 1000, df['Price(Dollar)'].max()]
price_labels = ['0-20', '21-50', '51-100', '101-200', '201-500', '501-1000', '1000+']
df['Price Bin'] = pd.cut(df['Price(Dollar)'], bins=price_bins, labels=price_labels, right=False)

df = df.drop('Shipment', axis=1)

display(df.head())

In [ ]:
print("Missing values before handling:\n", df.isnull().sum())


In [ ]:
Q1_price = df['Price(Dollar)'].quantile(0.25)
Q3_price = df['Price(Dollar)'].quantile(0.75)
IQR_price = Q3_price - Q1_price
lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price
df = df[(df['Price(Dollar)'] >= lower_bound_price) & (df['Price(Dollar)'] <= upper_bound_price)]


Q1_reviews = df['Number of  reviews'].quantile(0.25)
Q3_reviews = df['Number of  reviews'].quantile(0.75)
IQR_reviews = Q3_reviews - Q1_reviews
lower_bound_reviews = Q1_reviews - 1.5 * IQR_reviews
upper_bound_reviews = Q3_reviews + 1.5 * IQR_reviews
df = df[(df['Number of  reviews'] >= lower_bound_reviews) & (df['Number of  reviews'] <= upper_bound_reviews)]


print("\nNumber of duplicate rows before handling:", df.duplicated().sum())



In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
print("Missing values after handling:\n", df.isnull().sum())

In [ ]:
print("Number of duplicate rows after handling:", df.duplicated().sum())

In [ ]:
features = ['Price(Dollar)', 'Number of  reviews', 'Product Category', 'Shipment Location', 'Price Bin']
df_features = df[features].copy()

# one-hot encoding for categorical features
df_features = pd.get_dummies(df_features, columns=['Product Category', 'Shipment Location', 'Price Bin'], dummy_na=False)

display(df_features.head())

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
daily_sales = df.groupby('Order Date')['Price(Dollar)'].sum().reset_index()
display(daily_sales.head())


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(daily_sales['Order Date'], daily_sales['Price(Dollar)'])
plt.title('Daily Sales Trend')
plt.xlabel('Date')
plt.ylabel('Total Sales (Dollar)')
plt.grid(True)
plt.show()

Observations:


1.   The plot shows the daily total sales in dollars over a period from early 2019 to late 2022.
2.   There is a noticeable upward trend in sales over the years, although there are fluctuations.
3. Several significant spikes in sales are visible throughout the time period, indicating days with unusually high sales.
4. The sales appear to be seasonal, with some periods showing higher variability or consistent peaks.



In [ ]:
category_performance = df.groupby('Product Category').agg(
    Total_Sales=('Price(Dollar)', 'sum'),
    Average_Reviews=('Number of  reviews', 'mean')
).reset_index()

category_performance = category_performance.sort_values(by='Total_Sales', ascending=False)

display(category_performance)

In [ ]:

plt.figure(figsize=(10,6))
plt.bar(category_performance['Product Category'], category_performance['Total_Sales'])
plt.title('Total Sales by Product Category')
plt.xlabel('Product Category')
plt.ylabel('Total Sales (Dollar)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,6))
plt.bar(category_performance['Product Category'], category_performance['Average_Reviews'])
plt.title('Average Reviews by Product Category')
plt.xlabel('Product Category')
plt.ylabel('Average Number of Reviews')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


> we could identify product categories that have high total sales but relatively lower average reviews, which might indicate areas for product improvement or marketing focus. Conversely, categories with high average reviews but lower sales could benefit from increased visibility or promotional efforts.



In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Price(Dollar)', y='Number of  reviews', alpha=0.5)
plt.title('Price vs. Number of Reviews')
plt.xlabel('Price (Dollar)')
plt.ylabel('Number of Reviews')
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
sns.regplot(data=df, x='Price(Dollar)', y='Number of  reviews', scatter_kws={'alpha':0.5})
plt.title('Price vs. Number of Reviews with Regression Line')
plt.xlabel('Price (Dollar)')
plt.ylabel('Number of Reviews')
plt.grid(True)
plt.show()



> While there's a weak tendency for cheaper products to have more reviews on
average, the relationship is not a strong or simple one. Many other factors contribute to how many reviews a product receives.



In [ ]:
# Analyzing category trends over time
df['Order YearMonth'] = df['Order Date'].dt.to_period('M')

monthly_category_sales = df.groupby(['Order YearMonth', 'Product Category'])['Price(Dollar)'].sum().reset_index()

monthly_category_sales_pivot = monthly_category_sales.pivot_table(
    index='Order YearMonth',
    columns='Product Category',
    values='Price(Dollar)',
    fill_value=0
)

display(monthly_category_sales_pivot.head())


top_categories = category_performance.head(5)['Product Category'].tolist()

plt.figure(figsize=(15, 8))
for category in top_categories:
    if category in monthly_category_sales_pivot.columns:
        plt.plot(monthly_category_sales_pivot.index.astype(str), monthly_category_sales_pivot[category], label=category)

plt.title('Monthly Sales Trend for Top Product Categories')
plt.xlabel('Month-Year')
plt.ylabel('Total Sales (Dollar)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


**We can see that over the years men's shoes and men's clothing are getting new highs, increasing demand for these categories**

In [ ]:
print("Top Categories by Total Sales and Average Reviews:")
display(category_performance)



> Analysis of Dominant and Growing Categories:


 1. Based on the analysis, 'Men Shoes' and 'Men Clothes' are the dominant categories in terms of total sales and show a clear upward trend, indicating strong growth.
 2. Possible reasons for their dominance and growth could include: consistent demand for apparel and footwear, fashion trends, effective marketing strategies, or a wider variety of products available in these categories.

 3. Categories with relatively high average reviews but lower total sales (e.g., 'Toys', 'Car Accessories', 'Audio Video') might have highly satisfied customers but a smaller market size or less visibility.
 4. Conversely, categories with high total sales but relatively lower average reviews could indicate areas for product improvement or managing customer expectations.

In [ ]:
daily_sales = daily_sales.set_index('Order Date')
weekly_sales = daily_sales.resample('W')['Price(Dollar)'].sum()
weekly_sales = weekly_sales.fillna(0)
display(weekly_sales.head())

In [ ]:
weekly_sales = daily_sales.resample('W')['Price(Dollar)'].sum()
weekly_sales = weekly_sales.fillna(0)
display(weekly_sales.head())

In [ ]:
from prophet import Prophet

prophet_df = weekly_sales.reset_index()
prophet_df.columns = ['ds', 'y']

model = Prophet()

model.fit(prophet_df)

In [ ]:
future = model.make_future_dataframe(periods=52, freq='W')
forecast = model.predict(future)
display(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

In [ ]:
fig1 = model.plot(forecast)
plt.title('Weekly Sales Forecast')
plt.xlabel('Date')
plt.ylabel('Total Sales (Dollar)')
plt.show()

fig2 = model.plot_components(forecast)
plt.show()

## Observations from the Weekly Sales Forecast

The generated plots provide valuable insights into the weekly sales forecast:

1.  **Weekly Sales Forecast Plot:**
    *   The black dots represent the actual historical weekly sales data.
    *   The blue line shows the Prophet model's forecast, extending into the future.
    *   The shaded blue area represents the uncertainty interval (the range within which the actual sales are expected to fall with a certain probability).
    *   We can observe how well the model captures the historical trends and seasonality. The forecast line generally follows the pattern of the historical data and extends this pattern into the future.
    *   The uncertainty interval widens as we forecast further into the future, which is expected as the model's confidence decreases over longer time horizons.

2.  **Forecast Components Plot:**
    *   **Trend:** This plot shows the overall long-term trend in weekly sales. We can see a clear upward trend over the years, indicating increasing sales over time.
    *   **Yearly Seasonality:** This plot reveals the yearly seasonal pattern in sales. We can observe recurring peaks and troughs within each year, suggesting that sales are influenced by the time of year. The plot shows a strong peak towards the end of the year (likely due to holiday seasons) and lower sales in the earlier months.


In [ ]:
X = df_features.copy()
y = df['Price(Dollar)'].copy()

print("Missing values in features (X):\n", X.isnull().sum())
print("\nMissing values in target (y):\n", y.isnull().sum())


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

In [ ]:
import numpy as np

new_product_data = {
    'Price(Dollar)': [50],
    'Number of  reviews': [100],
    'Product Category_Audio Video': [False],
    'Product Category_Camera': [False],
    'Product Category_Car Accessories': [False],
    'Product Category_Laptop': [False],
    'Product Category_Men Clothes': [True],
    'Product Category_Men Shoes': [False],
    'Product Category_Mobile & Accessories': [False],
    'Product Category_Toys': [False],
    'Shipment Location_Bangladesh': [True],
    'Price Bin_0-20': [False],
    'Price Bin_21-50': [True],
    'Price Bin_51-100': [False],
    'Price Bin_101-200': [False],
    'Price Bin_201-500': [False],
    'Price Bin_501-1000': [False],
    'Price Bin_1000+': [False]
}


new_product_df = pd.DataFrame(new_product_data)


missing_cols = set(X_train.columns) - set(new_product_df.columns)
for c in missing_cols:
    new_product_df[c] = False  # or 0, depending on how get_dummies handles missing categories

new_product_df = new_product_df[X_train.columns]


predicted_price = model.predict(new_product_df)

print(f"Predicted Price for the new product: ${predicted_price[0]:.2f}")

In [ ]:
coefficients = model.coef_
feature_importance = pd.Series(coefficients, index=X_train.columns)

sorted_feature_importance = feature_importance.abs().sort_values(ascending=False)

print("Feature Importance (absolute value):\n")
display(feature_importance[sorted_feature_importance.index])

# Observations / Findings
*   The feature importance analysis revealed that 'Price' was overwhelmingly the most influential factor in predicting the outcome, with an importance score of 1.0.

*   The Linear Regression model achieved an R-squared score of 1.0 on the test data, along with extremely low Mean Absolute Error (7.51e-17) and Mean Squared Error (1.08e-31). This indicates a perfect fit to the data.

*   Other features, such as certain 'Product Category' and 'Price Bin' dummy variables, showed some level of influence, although significantly less than 'Price'.




## Summary














*   **Sales Trends:** There is a clear upward trend in overall sales from 2019 to 2022, with noticeable seasonal peaks, likely related to holiday seasons.
*   **Category Performance:** 'Men Shoes' and 'Men Clothes' are the dominant product categories in terms of total sales and exhibit strong growth over time. Categories like 'Toys', 'Car Accessories', and 'Audio Video' have relatively higher average reviews but lower total sales, suggesting potential for targeted marketing or market size limitations.
*   **Price and Reviews Relationship:** While there is a weak tendency for cheaper products to have more reviews, the relationship between price and the number of reviews is not strong, indicating other factors are more influential.
*   **Sales Forecasting:** The Prophet model effectively captured the historical weekly sales trends and seasonality, providing a forecast that shows continued growth and a strong yearly pattern.
*   **Price Prediction:** The Linear Regression model achieved a perfect R-squared score of 1.0 and extremely low error metrics. The feature importance analysis revealed that the 'Price' feature itself was overwhelmingly the most important predictor in this specific model setup.